In [ ]:
!uv pip install 'scanpy @ git+https://github.com/keller-mark/scanpy@af55e9d'
!uv pip install 'pylemur @ git+https://github.com/keller-mark/pyLemur@2f484b0'

In [1]:
from compasce import run_all, create_dask_client, normalize_basic, normalize_pearson_residuals, densmap, COMPASCE_KEY, compute_diffexp
from compasce.io import ComparativeData

In [2]:
from anndata import read_zarr, read_h5ad
import numpy as np

In [3]:
def get_adata():
    adata = read_h5ad("KPMP_PREMIERE_SC_version1.5_ForExplorer_withRC.032624.h5ad")

    should_subset = True
    if should_subset:
        # subset using random sample so that multiple sample groups are represented to enable comparison
        np.random.seed(1)
        obs_subset = np.random.choice(adata.obs.index.tolist(), size=100_000, replace=False).tolist()
        var_slice = slice(None)
        adata = adata[obs_subset, var_slice].copy()
        adata.layers["counts"] = adata.raw[obs_subset, var_slice].X.todense()
    else:
        adata.layers["counts"] = adata.raw.X.todense()
    adata.raw = None
    return adata

In [4]:
# For KPMP_PREMIERE....h5ad
sample_id_col = "SampleID"
sample_group_pairs = [
  ('diseasetype', ('Reference', 'AKI')),
  ('diseasetype', ('CKD', 'AKI')),
  ('diseasetype', ('Reference', 'CKD')),
]

In [ ]:
(cdata, ladata) = run_all(
    get_adata,
    zarr_path="data/kpmp_premiere.cdata.zarr",
    client=create_dask_client(memory_limit='2GB'),
    sample_id_col=sample_id_col,
    sample_group_pairs=sample_group_pairs,
)